In [2]:
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
from features_kappa_f import features_kappa
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


print("Helooo")
# Paths for original and filtered images
original_images_path = 'original'  # Folder with original images
filtered_images_path = 'filtered'  # Folder with median-filtered images

# Ensure the folders exist
if not os.path.exists(original_images_path):
    raise FileNotFoundError(f"Original images folder not found: {original_images_path}")
if not os.path.exists(filtered_images_path):
    raise FileNotFoundError(f"Filtered images folder not found: {filtered_images_path}")
# Fetch all image names
original_images = sorted([f for f in os.listdir(original_images_path) if f.endswith(('.jpg', '.png', '.tif', '.bmp'))])
filtered_images = sorted([f for f in os.listdir(filtered_images_path) if f.endswith(('.jpg', '.png', '.tif', '.bmp'))])

# Split filtered images into train and test
filtered_train, filtered_test = train_test_split(filtered_images, test_size=0.3, random_state=42)  # 30% for test

# Use all original images and only the training portion of filtered images for training
features = []
labels = []
c=0;
b=0;
# Original images (label 0)
for img_name in original_images:
    img_path = os.path.join(original_images_path, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    features.append(features_kappa(img))
    c=c+1
    print(c)
    labels.append(0)

# Filtered training images (label 1)
for img_name in filtered_train:
    img_path = os.path.join(filtered_images_path, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    features.append(features_kappa(img))
    b=b+1
    print(b)
    labels.append(1)

# Convert to NumPy
features = np.array(features)
labels = np.array(labels)

# Train the SVM
svm = SVC(kernel='rbf', C=1.0, gamma='scale')
svm.fit(features, labels)

# Now test only on the filtered_test set
filtered_test_features = []
filtered_test_labels = []

for img_name in filtered_test:
    img_path = os.path.join(filtered_images_path, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    filtered_test_features.append(features_kappa(img))
    filtered_test_labels.append(1)  # All are filtered

filtered_test_features = np.array(filtered_test_features)
filtered_test_labels = np.array(filtered_test_labels)

# Predict and evaluate
filtered_preds = svm.predict(filtered_test_features)
filtered_accuracy = accuracy_score(filtered_test_labels, filtered_preds)
print(f"Generalization Test Accuracy on unseen filtered images: {filtered_accuracy * 100:.2f}%")


Helooo
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Generalization Test Accuracy on unseen filtered images: 0.00%
